In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import random
from plotly.subplots import make_subplots
import warnings

# Load the data
file_path = 'dataset/HealthData.csv'
data = pd.read_csv(file_path)

# Preprocess the data
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['timestamp'])
data_cleaned = data.copy()
for col in ['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col], errors='coerce')
data_cleaned = data_cleaned.dropna()

# Calculate Heart Rate Variability (HRV)
data_cleaned['hrv'] = data_cleaned['heartrate'].diff()

# Correlation matrix
correlation_matrix = data_cleaned[['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']].corr()

# Summary Statistics
def generate_summary_stats():
    return {
        'Average Heart Rate (BPM)': round(data_cleaned['heartrate'].mean(), 2),
        'Average Blood Oxygen Level (%)': round(data_cleaned['blood_oxygen_level'].mean(), 2),
        'Average Body Temperature (°C)': round(data_cleaned['body_temperature'].mean(), 2),
        'Average Room Temperature (°C)': round(data_cleaned['room_temperature'].mean(), 2),
        'Average Hydration Levels (%)': round(data_cleaned['hydration_levels'].mean(), 2),
        'Total Falls Detected': int(data_cleaned['fall_detection'].sum()),
        'Overall Health Score': round(((data_cleaned['heartrate'].mean() * 0.3) +
                                      (data_cleaned['blood_oxygen_level'].mean() * 0.3) +
                                      (data_cleaned['body_temperature'].mean() * 0.2) +
                                      (data_cleaned['hydration_levels'].mean() * 0.2)), 2)
    }

summary_stats = generate_summary_stats()

# Healthy range comparison data
healthy_ranges = {
    'Heart Rate (BPM)': '60-100',
    'Blood Oxygen Level (%)': '95-100',
    'Body Temperature (°C)': '36.1-37.2',
    'Room Temperature (°C)': '18-21',
    'Hydration Levels (%)': '50-60'
}

# Dash App Init
app = dash.Dash(__name__)

# Shared sub-tabs generator
def shared_tabs(metric, include_trend=False):
    tabs = [
        dcc.Tab(label='Gauge Charts', children=[
            dcc.Graph(figure=go.Figure(
                data=[go.Indicator(
                    mode="gauge+number",
                    value=data_cleaned[metric].iloc[-1],
                    title={'text': f"{metric.replace('_', ' ').title()} Gauge"},
                    gauge={'axis': {'range': [None, data_cleaned[metric].max()]}}
                )]
            ))
        ]),
        dcc.Tab(label='Time Series Analysis', children=[
            dcc.Graph(figure=go.Figure(
                data=[go.Scatter(
                    x=data_cleaned['datetime'],
                    y=data_cleaned[metric],
                    mode='lines',
                    name=f"{metric} trend"
                )],
                layout=go.Layout(
                    title=f"{metric.replace('_', ' ').title()} Time Series Analysis",
                    xaxis_title='Time',
                    yaxis_title=metric
                )
            ))
        ])
    ]

    if include_trend:
        tabs.append(
            dcc.Tab(label='Trend Forecasting', children=[
                dcc.Graph(figure=go.Figure(
                    data=[go.Scatter(
                        x=data_cleaned['datetime'],
                        y=data_cleaned[metric].rolling(window=50).mean(),
                        mode='lines',
                        name='Trend Forecast',
                        line=dict(dash='dot')
                    )],
                    layout=go.Layout(
                        title=f"{metric.replace('_', ' ').title()} Forecast Trend",
                        xaxis_title='Time',
                        yaxis_title=metric
                    )
                ))
            ])
        )

    return tabs

# Summary layout with tabs
summary_tab_layout = dcc.Tabs([
    dcc.Tab(label='Summary Statistics', children=[html.Div([
        html.H3("Summary Statistics", style={'text-align': 'center'}),
        html.Ul([
            html.Li(f"Average Heart Rate (BPM): {summary_stats['Average Heart Rate (BPM)']}"),
            html.Li(f"Average Blood Oxygen Level (%): {summary_stats['Average Blood Oxygen Level (%)']}"),
            html.Li(f"Average Body Temperature (°C): {summary_stats['Average Body Temperature (°C)']}"),
            html.Li(f"Average Room Temperature (°C): {summary_stats['Average Room Temperature (°C)']}"),
            html.Li(f"Average Hydration Levels (%): {summary_stats['Average Hydration Levels (%)']}"),
            html.Li(f"Total Falls Detected: {summary_stats['Total Falls Detected']}"),
            html.Li(f"Overall Health Score: {summary_stats['Overall Health Score']}")
        ])
    ])]),
    dcc.Tab(label='Comparison with Healthy Ranges', children=[
        html.Table([
            html.Tr([html.Th("Metric"), html.Th("Your Average"), html.Th("Healthy Range")]),
            html.Tr([html.Td("Heart Rate (BPM)"), html.Td(summary_stats['Average Heart Rate (BPM)']), html.Td(healthy_ranges['Heart Rate (BPM)'])]),
            html.Tr([html.Td("Blood Oxygen Level (%)"), html.Td(summary_stats['Average Blood Oxygen Level (%)']), html.Td(healthy_ranges['Blood Oxygen Level (%)'])]),
            html.Tr([html.Td("Body Temperature (°C)"), html.Td(summary_stats['Average Body Temperature (°C)']), html.Td(healthy_ranges['Body Temperature (°C)'])]),
            html.Tr([html.Td("Room Temperature (°C)"), html.Td(summary_stats['Average Room Temperature (°C)']), html.Td(healthy_ranges['Room Temperature (°C)'])]),
            html.Tr([html.Td("Hydration Levels (%)"), html.Td(summary_stats['Average Hydration Levels (%)']), html.Td(healthy_ranges['Hydration Levels (%)'])])
        ])
    ]),
    dcc.Tab(label='Correlation Heatmap', children=[
        dcc.Graph(figure=ff.create_annotated_heatmap(
            z=correlation_matrix.values,
            x=list(correlation_matrix.columns),
            y=list(correlation_matrix.index),
            annotation_text=correlation_matrix.round(2).values,
            colorscale='Blues'))
    ]),
    dcc.Tab(label='Room vs Body Temperature', children=[
        dcc.Graph(figure=go.Figure([
            go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['room_temperature'], mode='lines', name='Room Temp'),
            go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['body_temperature'], mode='lines', name='Body Temp')
        ], layout=go.Layout(title="Room vs Body Temperature", xaxis_title='Time', yaxis_title='Temperature (°C)')))
    ])
])

# Epilepsy detection function
def detect_epilepsy(health_data):
    health_data = health_data.copy()
    window_size = 5
    health_data['heartrate_rolling_std'] = health_data['heartrate'].rolling(window=window_size).std()
    health_data['oxygen_rolling_std'] = health_data['blood_oxygen_level'].rolling(window=window_size).std()
    health_data['temp_rolling_std'] = health_data['body_temperature'].rolling(window=window_size).std()

    health_data['epilepsy_alert'] = False
    for i in range(len(health_data)):
        abnormal_metrics = 0
        if health_data.loc[i, 'heartrate_rolling_std'] > 30:
            abnormal_metrics += 1
        if health_data.loc[i, 'oxygen_rolling_std'] > 3:
            abnormal_metrics += 1
        if health_data.loc[i, 'temp_rolling_std'] > 0.5:
            abnormal_metrics += 1
        if abnormal_metrics >= 2:
            health_data.loc[i, 'epilepsy_alert'] = True
    return health_data

def create_epilepsy_detection_tab():
    alert_data = detect_epilepsy(data_cleaned)
    alert_count = alert_data['epilepsy_alert'].sum()
    alert_color = 'green' if alert_count == 0 else 'red'

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=alert_data['datetime'], y=alert_data['heartrate'], mode='lines', name='Heart Rate', line=dict(color='blue')))
    fig.add_trace(go.Scatter(
        x=alert_data[alert_data['epilepsy_alert']]['datetime'],
        y=alert_data[alert_data['epilepsy_alert']]['heartrate'],
        mode='markers', name='Possible Seizure',
        marker=dict(color='red', size=10, symbol='x')))

    fig.update_layout(title='Your Heart Rate with Possible Seizures Marked', xaxis_title='Time', yaxis_title='Heart Rate')

    return dcc.Tab(label='Epilepsy Detection', children=[
        html.Div([
            html.H2("Seizure Alert System", style={'textAlign': 'center'}),
            html.Div([
                html.H4("How This Works:"),
                html.P("This system watches for unusual changes in your:"),
                html.Ul([
                    html.Li("Heart rate"),
                    html.Li("Blood oxygen"),
                    html.Li("Body temperature")
                ]),
                html.P("When two or more of these change suddenly at the same time, it may indicate a possible seizure."),
                html.P("Important:", style={'color': 'red', 'fontWeight': 'bold'})
            ], style={'backgroundColor': '#f7f7f7', 'padding': 20}),
            html.Div([
                html.H3("Total Possible Seizure Alerts", style={'textAlign': 'center'}),
                html.H1(f"{alert_count}", style={'textAlign': 'center', 'color': alert_color}),
                html.P("alerts detected", style={'textAlign': 'center'})
            ], style={'border': f'2px solid {alert_color}', 'padding': 20, 'margin': '20px auto', 'width': '60%', 'borderRadius': 10}),
            dcc.Graph(figure=fig),
            html.H4("When Alerts Happened:", style={'textAlign': 'center'}),
            html.P(
                "No seizure alerts detected – this is good!" if alert_count == 0 else "Seizure alerts were detected – review the red X marks.",
                style={'textAlign': 'center', 'color': alert_color}
            ),
            html.Div([
                html.H4("What Should I Do?"),
                html.Ul([
                    html.Li("Take note of the X marks"),
                    html.Li("Recall how you felt during those times"),
                    html.Li("Show this to your doctor"),
                    html.Li("Call your doctor sooner if you see many alerts")
                ]),
                html.P("Remember: This system isn't perfect. Always trust how you feel over what the monitor says.",
                       style={'fontStyle': 'italic'})
            ], style={'backgroundColor': '#e8f4fb', 'padding': 20})
        ])
    ])


# App layout
app.layout = html.Div([
    html.H1("Health Monitoring Dashboard", style={'text-align': 'center'}),
    dcc.Tabs([
        dcc.Tab(label='Summary', children=[summary_tab_layout]),
        dcc.Tab(label='Heart', children=[
            dcc.Tabs(shared_tabs(metric='heartrate') + [
                dcc.Tab(label='Live Heart Rate Stream', children=[
                    dcc.Graph(id='live-heart-chart'),
                    dcc.Interval(id='interval-component', interval=2000, n_intervals=0)
                ])
            ])
        ]),
        dcc.Tab(label='Epilepsy', children=[
            dcc.Tabs(
            shared_tabs(metric='body_temperature', include_trend=True) + 
            [create_epilepsy_detection_tab()]
        )
    ]),
        dcc.Tab(label='Respiratory', children=[
            dcc.Tabs(shared_tabs(metric='blood_oxygen_level') + [
                dcc.Tab(label='Live Blood Oxygen Stream', children=[
                    dcc.Graph(id='live-oxygen-chart'),
                    dcc.Interval(id='interval-oxygen', interval=2000, n_intervals=0)
                ])
            ])
        ]),
        dcc.Tab(label='Mental Health', children=[html.Div("Mental health visualizations coming soon.")])
    ])
])

# Live heart chart callback
@app.callback(
    Output('live-heart-chart', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_heart_chart(n):
    df = data_cleaned.iloc[:n+1]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['datetime'],
        y=df['heartrate'],
        mode='lines+markers',
        name='Heart Rate',
        line=dict(color='red')
    ))
    fig.update_layout(title='Live Heart Rate Monitoring', xaxis_title='Time', yaxis_title='Heart Rate (BPM)')
    return fig

# Live blood oxygen callback
@app.callback(
    Output('live-oxygen-chart', 'figure'),
    Input('interval-oxygen', 'n_intervals')
)
def update_oxygen_chart(n):
    df = data_cleaned.iloc[:n+1]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['datetime'],
        y=df['blood_oxygen_level'],
        mode='lines+markers',
        name='Blood Oxygen Level',
        line=dict(color='blue')
    ))
    fig.update_layout(title='Live Blood Oxygen Level Monitoring', xaxis_title='Time', yaxis_title='Blood Oxygen Level (%)')
    return fig

# Run app
if __name__ == '__main__':
    app.run(debug=True, port=8052)


## Comprehensive Overview of the Health Monitoring Dashboard Code:

### 1. **Importing Required Libraries:**
   - **dash, dcc (dash_core_components), html (dash_html_components)**: Core libraries from Dash used to build the interactive dashboard for health monitoring.
   - **plotly.express (px) and plotly.graph_objs (go)**: Libraries from Plotly for creating interactive visualizations, including simple (express) and more detailed (graph_objs) plotting tools.
   - **plotly.subplots and figure factory**: Libraries for subplot layouts and heatmaps.
   - **pandas**: A library for data manipulation and analysis, essential for loading, cleaning, and processing health data from the CSV file.
   - **numpy**: A numerical library used to calculate Heart Rate Variability (HRV) by computing differences between heart rate measurements.
   - **warnings and random**: General utilities.

### 2. **Loading and Preprocessing the Data:**
   - **Loading the CSV File**: The code loads health monitoring data from a CSV file named `HealthData.csv`.
   - **Datetime Conversion**: The date and timestamp columns are combined into a single `datetime` column to support time-series analysis.
   - **Data Cleaning**: The key health metrics such as heart rate, blood oxygen level, body temperature, room temperature, and hydration levels are converted to numeric format. Any missing values are dropped to ensure data integrity.
   - **HRV Calculation**: Heart Rate Variability (HRV) is computed by calculating the difference between consecutive heart rate values (`data['heartrate'].diff()`).

### 3. **Calculating Key Metrics:**
   - **Summary Statistics**: 
     - Average Heart Rate (BPM)
     - Average Blood Oxygen Level (%)
     - Average Body Temperature (°C)
     - Average Room Temperature (°C)
     - Average Hydration Levels (%)
     - Total Falls Detected
   
   - **Overall Health Score**: This is calculated based on an average of key metrics (heart rate, blood oxygen level, body temperature), providing an overall view of the user’s health on the summary page.

   - **Healthy Standard Ranges**: Predefined healthy ranges for elderly individuals are included to provide a comparison against the user’s actual data.

### 4. **Initializing the Dash App:**
   - `app = dash.Dash(__name__)`: Initializes the Dash app.
   - The `app.layout` defines the structure of the dashboard and how the user interacts with different components (visualizations, tabs).

### 5. **Creating Interactive Visualizations:**
   Each visualization provides an intuitive way for elderly users to monitor and understand their health metrics:

   #### A. **Gauge Charts for Key Health Metrics**:
   - **Heart Rate (BPM)**, **Blood Oxygen Level (%)**, and **Hydration Levels (%)** are represented using gauge charts. These charts display user metrics compared to healthy standard ranges, with color-coded indicators (green = healthy, yellow = caution, red = alert).

   #### B. **Summary Page**:
   - Displays key health statistics like heart rate, blood oxygen levels, body temperature, room temperature, hydration levels, and the total number of falls detected.
   - The overall health score is also displayed, giving users a quick snapshot of their health condition.

   #### C. **Time-Series Analysis**:
   - **Graph Description**: This visualization shows changes in Heart Rate (BPM), Blood Oxygen Level (%), and Body Temperature (°C) over time.
   - **Explanation**: The blue line represents heart rate, the red line shows blood oxygen, and the green line is for body temperature, allowing the user to easily track trends in their health metrics.

   #### D. **Box Plot: Heart Rate by Fall Detection**:
   - **Graph Description**: A box plot comparing heart rate when falls are detected (orange) versus when no falls are detected (blue).
   - **Explanation**: Higher heart rates during falls may indicate physical activity or stress.

   #### E. **HRV Distribution (Heart Rate Variability)**:
   - **Graph Description**: This histogram shows the distribution of Heart Rate Variability (HRV), which measures how much the user’s heart rate changes over time.
   - **Explanation**: Higher HRV typically indicates relaxation, while lower HRV could signal stress or discomfort.

   #### F. **Room and Body Temperature Comparison**:
   - **Graph Description**: This graph shows how the room temperature (blue line) compares to the body temperature (orange line) over time.
   - **Explanation**: It helps users see if fluctuations in room temperature might affect their body temperature and overall comfort.

   #### G. **Correlation Heatmap**:
   - **Graph Description**: A heatmap showing how different health metrics (e.g., heart rate, blood oxygen level, body temperature, hydration levels) correlate with each other.
   - **Explanation**: A positive correlation means two metrics tend to increase together, while a negative correlation means one metric decreases as the other increases.

   #### H. Live Heart Rate Stream:
-**Graph Description**: This visualization simulates a real-time line chart that streams heart rate (BPM) data point by point using Dash’s dcc.Interval component. Each new point is displayed every two seconds, mimicking how a live sensor feed would appear.
-**Explanation**: The heart rate values are appended progressively from the dataset to emulate continuous monitoring. This dynamic feature provides a foundation for future integration with real-time APIs like ThingSpeak or other IoT devices.

   #### I. Live Blood Oxygen Stream:
-**Graph Description**: A real-time streaming graph tracks changes in blood oxygen levels over time using a clean, dynamic line plot interface.
-**Explanation**: This visualization mirrors the heart rate stream format for design consistency. It updates every 2 seconds, helping users monitor blood oxygen trends live without visual clutter. The minimalistic approach supports clarity while maintaining clinical utility for detecting gradual oxygen saturation changes.

   #### J. Trend Forecasting (Predictive Analysis):
-**Graph Description**: A multi-panel subplot visualization displaying rolling trend forecasts for heart rate, blood oxygen levels, and hydration.
-**Explanation**: Each subplot highlights raw data alongside a smoothed 50-point moving average line, offering a clearer view of long-term health trends. This approach aids in identifying gradual changes and potential risk patterns, helping preemptively flag deterioration before it becomes critical. The split format allows for targeted insights into each metric over time, boosting interpretability and focus.

   #### K. Epilepsy Detection Alerts (Experimental)
-**Graph Description**: A line chart of heart rate over time with red "X" markers indicating potential seizure episodes based on simultaneous anomalies across multiple metrics. Accompanied by a seizure count summary and user guidance interface.

-**Explanation**: This draft feature detects potential seizures by monitoring sudden changes (via rolling standard deviation) in heart rate, blood oxygen, and temperature. If two or more metrics spike concurrently, a possible seizure is flagged. The alert system includes visual cues and educational context but is not diagnostic. It’s designed to support self-reflection and promote informed conversations with healthcare professionals.




### 6. **Comparison with Healthy Ranges:**
   - **Page Description**: This page provides a side-by-side comparison between the user’s average metrics and the healthy standard ranges for elderly individuals. It helps users understand how their health metrics compare with optimal values.

### 7. **Creating the Layout**:
   - The layout is defined using `html.Div` to organize the structure of the dashboard. Users can navigate between different tabs using `dcc.Tabs`.
   - Each tab contains a unique visualization (gauge charts, time-series analysis, box plot, HRV distribution, room vs. body temperature comparison, and correlation heatmap).

### 8. **Running the App**:
   - `if __name__ == '__main__':` ensures that the Dash app runs only when the script is executed directly.
   - `app.run_server(debug=True)` starts the web server, allowing users to interact with the dashboard in their browser.

### Summary of Key Features:
   - **Gauge Charts**: Visualize heart rate, blood oxygen level, and hydration against healthy ranges with easy-to-read color codes.
   - **Comparison Page**: Compares the user’s metrics against healthy standard ranges for elderly individuals, providing a clear understanding of their health status.
   - **Time-Series Analysis**: Shows how key metrics change over time, allowing users to track trends.
   - **Box Plot**: Compares heart rate during falls versus non-fall situations.
   - **HRV Histogram**: Displays fluctuations in heart rate, indicating stress or relaxation levels.
   - **Room vs. Body Temperature**: Compares room temperature with body temperature to assess comfort.
   - **Correlation Heatmap**: Helps users understand how different health metrics relate to each other.
   - **Live Heart Rate Stream**: A real-time dynamic line chart that visualizes heart rate data as if it were streaming from a live sensor. Updates every 2 seconds.
   - **Live Blood Oxygen Stream**: A real-time dynamic line chart that visualizes blood oxygen level data as if it were streaming from a live sensor. Updates every 2 seconds and mirrors the heart rate stream for visual consistency.
    - **Trend Forecasting**: A 3-panel subplot displaying heart rate, blood oxygen, and hydration levels with real-time trendlines using 50-point moving averages. Designed to reveal long-term shifts and emerging health risks through smoothed visual patterns.
    - **Epilepsy Detection**: A seizure alert prototype that overlays red “X” markers on heart rate charts when sudden changes occur across heart rate, oxygen, and temperature. Designed as an early warning tool using rolling standard deviation thresholds to flag multi-metric anomalies that may indicate seizures. 

This dashboard is designed with elderly users in mind, offering intuitive visualizations and explanations to help them monitor their health and understand key trends.
